In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras import Sequential

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
train      = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/train_data.csv")
test       = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/test_data.csv")
submission = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/sample_submission.csv")
topic_dict = pd.read_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/dataset/topic_dict.csv")

In [ ]:
!pip install hanja

# 전처리

## 1. 한자처리

In [12]:
def chinese_etc(x):
    x = re.sub('→', '에서', x)
    x = re.sub('↑', '증가', x)
    x = re.sub('↓', '감소', x)
    x = re.sub('美', '미국', x)
    x = re.sub('韓', '한국', x)
    x = re.sub('日', '일본', x)
    x = re.sub('中', '중국', x)
    x = re.sub('英', '영국', x)
    x = re.sub('獨', '독일', x)
    x = re.sub("伊", "이탈리아", x)  
    x = re.sub("佛", "프랑스", x)
    x = re.sub('北', '북한', x)
    x = re.sub("亞", "아시아", x)
    x = re.sub("印", "인도", x)
    x = re.sub('與', '여당 ', x)
    x = re.sub('野', '야당 ', x)
    x = re.sub('黨', '당', x)
    x = re.sub('尹', '윤석열', x)
    x = re.sub('朴', '박근혜', x)
    x = re.sub('文', '문재인', x)
    x = re.sub('靑', '청와대', x)
    x = re.sub('反', '반대 ', x)
    x = re.sub("非", "아닌 ", x)  # 정치에선 반대랑 같은 의미로 쓰이는데 비청년에서는 반대로 해석하기 좀 그럼
    x = re.sub('親', '친', x)
    x = re.sub('南', '남한', x)
    x = re.sub('軍', '군대', x)
    x = re.sub('檢', '검찰', x)
    x = re.sub("新", "새로운 ", x)
    x = re.sub("弗", "달러", x)
    x = re.sub("對", "대결", x)
    x = re.sub("男", "남자", x)
    x = re.sub('女', '여자 ', x)
    x = re.sub('故', '고인', x)
    x = re.sub('前', '이전 ', x)
    x = re.sub('株', '주식 ', x)
    x = re.sub("發", "발생", x)
    x = re.sub("重", "중공업", x) 
    x = re.sub('家', '집안', x)
    x = re.sub('丁', '정세균', x) 
    x = re.sub('乙', '을', x) 
    x = re.sub('人', '명', x)
    x = re.sub('代', '대', x)
    x = re.sub('先', '먼저 ', x)
    x = re.sub('全', '전체', x)  # '전'세계에서는 조금 어색
    x = re.sub('前', '이전', x)
    x = re.sub('史', '역사', x)
    x = re.sub('和', '화합', x)  
    x = re.sub("外", "외", x)  # 외 -> 불용어  
    x = re.sub("大", "대학교", x)
    x = re.sub('妻', '부인', x)
    x = re.sub("子", "아들", x)
    x = re.sub("孫", "손학규", x) # 성씨 손 -> train에선 다 손학규로 쓰였는데 다른 손씨 (우리금융회장)으로 쓰이면? 
    x = re.sub('安', '안철수', x) # 안철수로 많이 쓰이지만 but 안(내부) 으로도 쓰임 -> 어떻게하지
    x = re.sub('富', '부', x)
    x = re.sub('對', '대', x)  # 대결, 맞서다 
    x = re.sub('山', '산', x)
    x = re.sub('崔', '최순실', x)  
    x = re.sub('州', '주', x)  # 외국 지역 단위 주 -> 삭제해도 될 듯?
    x = re.sub('市', '도시', x)
    x = re.sub('後', '이후', x)
    x = re.sub("情", "정", x) 
    x = re.sub('惡', '악', x)
    x = re.sub('曺', '조국', x)  # 법무부장관
    x = re.sub('月', '월', x)
    x = re.sub("核", "핵", x)
    x = re.sub("案", "안건", x)
    x = re.sub("機", "비행기", x)
    x = re.sub("氣", "기", x)
    x = re.sub('江', '강', x)
    x = re.sub('法', '법', x)
    # x = re.sub('洪', '홍', x)  # 성씨 -> 머투 홍선근 회장, 홍준표 정치인
    x = re.sub('湖', '호수', x)
    x = re.sub("潘", "반기문", x)
    x = re.sub("無", "없다", x) 
    x = re.sub('燈', '등', x) # 등불 -> 삭제? 
    x = re.sub('父', '아버지', x)
    x = re.sub('王', '왕', x)
    x = re.sub('現', '현재', x)
    x = re.sub('申', '신태용', x) # 축구감독, test에서는 다른 신씨로 쓰이면? 
    x = re.sub('知', '', x)  # 삭제 (앞에다 음 써놓음)
    x = re.sub('硏', '연구소', x)
    x = re.sub('社', '회사', x)  # 社告는 삭제해도 될 듯 (속보 같은 것)
    x = re.sub('禹', '우병우', x)
    x = re.sub("秋", "추미애", x)  # test에서 다른 추씨 나오면?
    x = re.sub("稅", "세", x) # 세금
    x = re.sub('臺', '대만', x)
    x = re.sub("色", "색", x)
    x = re.sub("萬", "인도", x)
    x = re.sub('號', '호', x)  # ooo호
    x = re.sub('行', '행', x)
    x = re.sub('詩', '시', x)
    x = re.sub('賞', '상', x)
    x = re.sub('趙', '조', x)  # 조윤선 -> test에서 다른 조씨 쓰이면? 
    x = re.sub('車', '자동차', x)
    x = re.sub('道', '도로', x)
    x = re.sub('選', '', x)  # 삭제해도 괜춘
    x = re.sub("鄭", "정진석", x)  
    x = re.sub('金', '김정은', x)  # 김씨가 너무 많음, 김정은으로 젤 많이 쓰인 듯 함
    if re.search("[가-핳]銀", x):
        x = re.sub("銀", "은행", x)
    x = re.sub('銀', '은메달', x)
    x = re.sub('銅', '동메달', x)
    x = re.sub('阿', '아프리카', x)
    x = re.sub("院", "국회 원", x)  # 국회 원 구성하다
    x = re.sub("黃", "황교안", x)
    x = re.sub("黨", "당", x)
    x = re.sub("蘭", "", x) # 난초 -> 삭제해도 괜춘
    x = re.sub("盧", "노무현", x)
    x = re.sub("樂", "", x)  # 락 -> 삭제해도 괜춘
    x = re.sub("年", "연간 ", x)  # 1년
    x = re.sub("劉", "유승민", x)
    # x = re.sub("李", "이 씨", x) # 이씨 너무 많음 -> train 4명 다 다른 인물 지칭
    x = re.sub("社告", "", x)  # 속보같은 단어들
    x = re.sub("書香萬里", "", x)
    x = re.sub("破七", "", x) # 포치: 옆에 한글로 써있음
    x = re.sub("氷魚", "", x)  # 빙어 :옆에 한글로 써있음 
    x = re.sub("不備", "", x)  # 불비 : 옆에 한글로 써있음
    x = re.sub("令和", "", x)  # 일본 연호 : 삭제 무방
    # x = hanja.translate(x, 'substitution')
    return x 



## 2. 동의어치환

In [6]:
def same_meaning(x):
    x = re.sub("스마트폰", "폰", x)
    x = re.sub("휴대폰", "폰", x)
    x = re.sub('갤', '갤럭시', x)
    x = re.sub("lg 30", "LG 폰", x)
    x = re.sub('LG G3', 'LG 폰', x)  # lg폰으로 ?
    x = re.sub('갤럭시 노트', '갤럭시', x)
    x = re.sub("갤럭시 10", "갤럭시", x)
    x = re.sub("삼성 노트", "갤럭시", x)
    # x = re.sub('삼성전자', '삼성', x)
    x = re.sub("삼성 전자", "삼성", x)
    x = re.sub("인공 지능", "인공지능", x)
    x = re.sub("갤럭시 폴드", "갤럭시", x)
    x = re.sub('카카오톡', '카톡', x)
    x = re.sub("휴대폰", "폰", x)
    # x = re.sub("패드", "태블릿", x)
    x = re.sub('지난해', '전년', x)
    x = re.sub("금융감독원", "금감원", x)
    x = re.sub("리스크", "위험", x)
    x = re.sub('테크', '기술', x)
    x = re.sub("베스트", "최고", x)  # 베스트셀러는 포함 안되겠지
    x = re.sub("한해", "연간", x)
    x = re.sub("대한민국", "한국", x)
    x = re.sub('남한', '한국', x)
    x = re.sub("온도", "기온", x)
    x = re.sub("갤럭시 폴드", "갤럭시", x)
    x = re.sub('카카오톡', '카톡', x)
    x = re.sub("대학교", "대학", x)
    x = re.sub('스토어', '매장', x)
    x = re.sub("점포", "매장", x)
    x = re.sub("고등학교", "고교", x)
    x = re.sub("대법원", "대법", x)
    x = re.sub('시내버스', '버스', x)
    x = re.sub("페스티벌", "축제", x)
    x = re.sub("제주도", "제주", x)
    x = re.sub('경기도', '경기', x) # 시합 경기랑 헷갈리려나
    x = re.sub("강원도", "강원", x)
    x = re.sub('전라도', '전라', x)
    x = re.sub("트렌드", "유행", x)
    x = re.sub('서울시', '서울', x)
    x = re.sub("페북", "페이스북", x)
    x = re.sub('군대', '군', x)
    return x


## 3. 불필요한 단어 제거

In [7]:
def word_delete(title): 
    delete_words = ["1보", "2보", "3보", "4보", "5보", "속보", "사진", "게시판", "사진", "게시판", "주말 N", "QA", "그래픽", "신간", 
                    "위클리", "주간 화제의 뉴스", "카드뉴스", "팩트체크"]

    for i in delete_words:
        if title.endswith(i) or title.startswith(i):
            title = title.replace(i, "")
    if title.endswith("종합"):
            title = title[:-2]
    title = title.upper() 
    return title

In [8]:
# ------ 코드분석 (1) -------
def clean_text(sent):
  sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
  return sent_clean

# ------ 코드분석 (2) -------
train["cleaned_title"] = train["title"].apply(lambda x : clean_text(x))
test["cleaned_title"]  = test["title"].apply(lambda x : clean_text(x))

train_text = train["cleaned_title"].tolist() # .tolist() : series 형식으로 된 데이터를 list로
test_text = test["cleaned_title"].tolist()
train_label = np.asarray(train.topic_idx) # np.asarray() : series 형식으로 된 데이터를 array로

# ------ 코드분석 (3) -------
tfidf = TfidfVectorizer(analyzer='word', sublinear_tf=True, ngram_range=(1, 2), max_features=150000, binary=False)

tfidf.fit(train_text)

train_tf_text = tfidf.transform(train_text).astype('float32')
test_tf_text  = tfidf.transform(test_text).astype('float32')

## 4. 최종 전처리 파일

In [13]:
def word_eda(title):
    title = chinese_etc(title)
    title = same_meaning(title)
    # title = stopwords(title)
    title = word_delete(title)
    title = clean_text(title)
    return title

In [14]:
train["title"] = train["title"].apply(word_eda)
test["title"] = test['title'].apply(word_eda)

In [15]:
train.to_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/preprocessed_dataset/ppc_train.csv")
test.to_csv("/content/drive/MyDrive/github_commit/sooktat_DL_project/preprocessed_dataset/ppc_test.csv")